<a href="https://colab.research.google.com/github/aashitadutta/FinancialNarrativeSummarization/blob/main/SentenceRank_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

### importing the necessary libraries

from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
import networkx as nx
import os
import math

In [ ]:
 import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
inp_data="/content/drive/MyDrive/lstm-input/"
out_data="/content/drive/MyDrive/lstm-summ/"

datasets={"text":inp_data,"summ":out_data}

data_categories=["training","validation","test"]

data={"articles":[],"summaries":[]}

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
!pip install Rouge

In [ ]:
!unzip training.zip

In [ ]:
def parsetext(dire,category,filename):
    with open("%s/%s"%(dire,filename),'r',encoding="Latin-1") as readin:
        print("file read successfully")
        text=readin.read()
    return text.lower()


def load_data(dire,category):
    for (root,dirs,files) in os.walk(dire):
         filenames=sorted(files)
    return filenames


def cleantext(text):
    text=re.sub(r"what's","what is ",text)
    text=re.sub(r"it's","it is ",text)
    text=re.sub(r"\'ve"," have ",text)
    text=re.sub(r"i'm","i am ",text)
    text=re.sub(r"\'re"," are ",text)
    text=re.sub(r"n't"," not ",text)
    text=re.sub(r"\'d"," would ",text)
    text=re.sub(r"\'s","s",text)
    text=re.sub(r"\'ll"," will ",text)
    text=re.sub(r"can't"," cannot ",text)
    text=re.sub(r" e g "," eg ",text)
    text=re.sub(r"e-mail","email",text)
    text=re.sub(r"9\\/11"," 911 ",text)
    text=re.sub(r" u.s"," american ",text)
    text=re.sub(r" u.n"," united nations ",text)
    text=re.sub(r"\n"," ",text)
    text=re.sub(r":"," ",text)
    text=re.sub(r"-"," ",text)
    text=re.sub(r"\_"," ",text)
    text=re.sub(r"\d+"," ",text)
    text=re.sub(r"[$#@%&*!~?%{}()]"," ",text)
    
    return text

def printArticlesum(k):
    print("---------------------original sentence-----------------------")
    print("-------------------------------------------------------------")
    print(data["articles"][k])
    print("----------------------Summary sentence-----------------------")
    print("-------------------------------------------------------------")
    print(data["summaries"][k])
    return 0


def announcedone():
    duration=2000
    freq=440
    ws.Beep(freq,duration)


f1_score1=[]
precision1=[]
recall1=[]
f1_score2=[]
precision2=[]
recall2=[]
f1_scoreL=[]
precisionL=[]
recallL=[]
filenames1=load_data(datasets["text"],data_categories[0])
filenames2=load_data(datasets["summ"],data_categories[0])
print(filenames1)
print(filenames2)

"""----------load the data, sentences and summaries-----------"""
data["articles"]=[]
data["summaries"]=[]
k=0
#while k<len(filenames1):
while k<10:
 try:
    print(filenames1[k])
    data["articles"].append(parsetext(datasets["text"],data_categories[0],"%s"%filenames1[k]))
 except Exception as e:
    data["articles"].append("Could not read")
    print(e)
 try:
   print(filenames2[k])
   data["summaries"].append(parsetext(datasets["summ"],data_categories[0],"%s"%filenames2[k]))
 except Exception as e:
   data["summaries"].append("Could not read")
   print(e)
 
 reference=""
 reference=data["summaries"][k]
 print("number of words in reference summary",len(reference))
 dict = {}
 s = ""
 for a in data['articles'][k]:
      s += a
 print("number of words in text file",len(s))
 
 # print s
 s = s.lower()
 # print s

 sentences = sent_tokenize(s)
 print("number of sentences in text file",len(sentences))
 # print sentences

 final = []

 for s in sentences:
      temp = re.sub("[^a-zA-Z0-9]"," ",s)
      temp = temp.lower()
      final.append(temp)
      dict[temp] = s
 # printfinal
 def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)




 def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
    return similarity_matrix
 stop_words = stopwords.words('english')
    # Step 2 - Generate Similary Martix across sentences
 sentence_similarity_martix = build_similarity_matrix(final, stop_words)

    # Step 3 - Rank sentences in similarity martix
 sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
 scores = nx.pagerank(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
 ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(final)), reverse=True)    
 model_out=""
 for i in range(5):
     model_out+=dict[ranked_sentence[i][1]]
     #print(dict[ranked_sentence[i][1]])
 from rouge import Rouge
 rouge = Rouge()
 print("number of words in the model summary",len(model_out))
 scores = rouge.get_scores(model_out, reference)
 scores_rouge1 = [score['rouge-1']['r'] for score in scores]
 scores_rouge2 = [score['rouge-2']['r'] for score in scores]
 scores_rougeL = [score['rouge-l']['r'] for score in scores]                 
 recall1.append(scores_rouge1[0])
 recall2.append(scores_rouge2[0])
 recallL.append(scores_rougeL[0])
 scores_rouge1 = [score['rouge-1']['p'] for score in scores]
 scores_rouge2 = [score['rouge-2']['p'] for score in scores]
 scores_rougeL = [score['rouge-l']['p'] for score in scores]                 
 precision1.append(scores_rouge1[0])
 precision2.append(scores_rouge2[0])
 precisionL.append(scores_rougeL[0])
 scores_rouge1 = [score['rouge-1']['f'] for score in scores]
 scores_rouge2 = [score['rouge-2']['f'] for score in scores]
 scores_rougeL = [score['rouge-l']['f'] for score in scores]                 
 f1_score1.append(scores_rouge1[0])
 f1_score2.append(scores_rouge2[0])
 f1_scoreL.append(scores_rougeL[0])
 k=k+1

print("the average f1_score for rouge-1 is",math.fsum(f1_score1)/len(f1_score1))
print("the average precision for rouge-1 is",math.fsum(precision1)/len(precision1))
print("the average recall for rouge-1 is",math.fsum(recall1)/len(recall1))
print("the average f1_score for rouge-2 is",math.fsum(f1_score2)/len(f1_score2))
print("the average precision for rouge-2 is",math.fsum(precision2)/len(precision2))
print("the average recall for rouge-2 is",math.fsum(recall2)/len(recall2))
print("the average f1_score for rouge-L is",math.fsum(f1_scoreL)/len(f1_scoreL))
print("the average precision for rouge-L is",math.fsum(precisionL)/len(precisionL))
print("the average recall for rouge-L is",math.fsum(recallL)/len(recallL))
 


 

 

['102.txt', '104.txt', '105.txt', '106.txt', '108.txt', '109.txt', '110.txt', '111.txt', '118.txt', '128.txt', '129.txt', '130.txt', '131.txt', '132.txt', '148.txt', '149.txt', '150.txt', '151.txt', '152.txt', '153.txt', '155.txt', '156.txt', '159.txt', '160.txt', '161.txt', '163.txt', '168.txt', '169.txt', '17.txt', '170.txt', '173.txt', '174.txt', '181.txt', '19.txt', '205.txt', '206.txt', '214.txt', '225.txt', '227.txt', '228.txt', '229.txt', '230.txt', '64.txt', '65.txt', '66.txt', '67.txt', '72.txt', '73.txt', '92.txt', '93.txt']
['102.txt', '104.txt', '105.txt', '106.txt', '108.txt', '109.txt', '110.txt', '111.txt', '118.txt', '128.txt', '129.txt', '130.txt', '131.txt', '132.txt', '148.txt', '149.txt', '150.txt', '151.txt', '152.txt', '153.txt', '155.txt', '156.txt', '159.txt', '160.txt', '161.txt', '163.txt', '168.txt', '169.txt', '17.txt', '170.txt', '173.txt', '174.txt', '181.txt', '19.txt', '205.txt', '206.txt', '214.txt', '225.txt', '227.txt', '228.txt', '229.txt', '230.txt'

In [ ]:
print(f1_score1)
print(f1_score2)
print(f1_scoreL)

[0.37414965497593594, 0.26897213805210207, 0.31940575223290396, 0.23773898171520497, 0.26298487486199135, 0.3126385762677667, 0.2532832972505096, 0.33461046821795043, 0.1285444196575914, 0.3193851360143403]
[0.07356947740112439, 0.04042348000385398, 0.09302325131706242, 0.029975016665352308, 0.060566158445080974, 0.03555555089355617, 0.026315784661107355, 0.06521738698252265, 0.015180261881043246, 0.056458506657837144]
[0.22963951447701528, 0.2256809292555528, 0.2560296798614903, 0.16293929262481, 0.18131101398901756, 0.16867469398880677, 0.15607984996439422, 0.18104667154152568, 0.1137123700656594, 0.181818176840249]


In [ ]:
from rouge import Rouge

In [ ]:
rouge = Rouge()

In [ ]:
print(len(model_out))
print(len(reference))

5679
5638


In [ ]:
rouge.get_scores([model_out], [reference], avg=True)

{'rouge-1': {'f': 0.4349334825676105,
  'p': 0.4253393665158371,
  'r': 0.44497041420118344},
 'rouge-2': {'f': 0.09264620229843894,
  'p': 0.09060022650056625,
  'r': 0.0947867298578199},
 'rouge-l': {'f': 0.27681660399659974,
  'p': 0.27586206896551724,
  'r': 0.2777777777777778}}